<a href="https://colab.research.google.com/github/chandan110791/Agentic_AI_Playground/blob/main/1_hr_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Libraries
from langchain_core.tools import StructuredTool

In [4]:
def climate_details(city:str)->None:
  print(f"city{city} details are cold")

In [5]:
from typing import Dict
from typing import List,Any
from dataclasses import dataclass
from pydantic import BaseModel

class ToolSpec(BaseModel):
    tool:StructuredTool = None
    desc:str = None
    name:str= None
    args:Dict[str,Any]=None

class ToolRegistery:
  def __init__(self) -> None:
    self.toolRegistery:Dict[str,ToolSpec]={}

  def update_registery(self,toolToBeAdded:ToolSpec) ->None:
    self.toolRegistery[toolToBeAdded.name]=toolToBeAdded

  def retrieve_all_tools(self)->List[ToolSpec]: ## Fix this
    return list(self.toolRegistery.values())

  def retrieve_tools(self,name:str)->StructuredTool:
    return self.toolRegistery[name].tool




In [1]:
!pip install langchain_huggingface
!pip install langchain_community
!pip install faiss-cpu
!pip install sentence-transformers


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
                      ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 95, in resolve
    result = self._result = resolver.resolve(
                            ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [18]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from typing import Optional
from langchain_huggingface import HuggingFaceEmbeddings
from dataclasses import dataclass
class Retriever(BaseModel):
    name:str
    similarity:float
    distance:float

class ToolRetriever:
  def __init__(self):
    self.result:str=None
    self.vectorStore:Optional[FAISS]=[]
    self.embeddings=HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2",encode_kwargs={"normalize_embeddings":True})

  def update_rag_registery(self,tool_reg:ToolRegistery)->None:
    docList:List[Document]=[]

    for tool_spec in tool_reg.retrieve_all_tools():
      name = tool_spec.name
      desc = tool_spec.desc
      text= f"name:{name} \n desc:{desc}"
      docList.append(Document(page_content=text,metadata={"tool_spec_name":name,"desc":desc}))

    self.vectorStore = FAISS.from_documents(docList,self.embeddings)

  def retrive_from_rag(self,query:str,top_k:int=5)->ToolSpec:
      res = self.vectorStore.similarity_search_with_score(query,k=top_k)
      out:List[Retriever]=[]
      for doc,dist in res:
        retriver = Retriever(name= doc.metadata["tool_spec_name"],distance=dist,similarity=(1/1+float(dist)))
        out.append(retriver)
      out.sort(key = lambda x: x.similarity,reverse=True)
      return out[0]




In [7]:
!pip install langgraph


In [16]:
from typing import TypedDict,Dict,Any
class AgentTool(TypedDict):
    candidates:List[ToolSpec]=None
    chosen:ToolSpec=None
    result:str=None
    error:str=None
    userQUery:str=None
    args:Dict[str,any]=()

def retrive_tool_agent(state: Dict,registery:ToolRegistery,toolretriever:ToolRetriever):
    retriever=toolretriever.retrive_from_rag(state["userQUery"],top_k=5)
    tool=registery.retrieve_tools(retriever.name)
    print(f"tool is {tool.name} and its args are {tool.args}")
    return {"chosen": tool}

def execute_tool_agent(state: Dict):
      args=state["args"]
      try:
        tool = state["chosen"]
        result = tool.invoke(args)
        return {"result":result}
      except Exception as e:
        print(f"return result as {e}")
        return {"error":e}

from langgraph.graph import StateGraph,START,END
def build_graph(agenttool:AgentTool,registery:ToolRegistery,toolretriever:ToolRetriever)-> StateGraph:
  g = StateGraph(AgentTool)

  g.add_node("retrive_tool_agent",lambda s :retrive_tool_agent(s,registery,toolretriever))
  g.add_node("execute_tool_agent",lambda s : execute_tool_agent(s))

  g.add_edge(START,"retrive_tool_agent")
  g.add_edge("retrive_tool_agent","execute_tool_agent")
  g.add_edge("execute_tool_agent",END)


  return g.compile()



In [17]:
def main():
  #declare the tool
  CLIMATE_TOOL = StructuredTool.from_function(name="climate_details",description="get climate details , args is city str",func=climate_details)
  # update the Tool Registery
  toolRegistery = ToolRegistery()
  # Correctly pass arguments matching ToolSpec's fields, deriving name and desc
  toolspec = ToolSpec(tool=CLIMATE_TOOL, name=CLIMATE_TOOL.name, desc=CLIMATE_TOOL.description, args={"city":"str"})
  toolRegistery.update_registery(toolspec)

  #UPdate the retriever
  toolRetriever = ToolRetriever()
  toolRetriever.update_rag_registery(toolRegistery)

  #Create the Agenttool
  agent_1 = AgentTool({"candidates":[],
    "chosen":[],
    "result":[],
    "error":[],
    "userQUery":"give me city for dubai",
    "args":{"city":"Dubai"}})

  #buildgraph

  g_main = build_graph(agenttool=agent_1,registery=toolRegistery,toolretriever=toolRetriever)

  g_main.invoke(agent_1)

  #invoke

if __name__== "__main__":
  main()

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


tool is climate_details and its args are {'city': {'title': 'City', 'type': 'string'}}
cityDubai details are cold


In [10]:
|# This cell is now empty as sentence-transformers installation has been moved.